import library

In [2]:
#import library
import pandas as pd
import numpy as np
import re
import csv
import nltk
from string import punctuation
from sklearn.model_selection import train_test_split
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import re
import csv
import string
import scipy.stats as st
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yudha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


mempersiapkan data set


In [3]:
#Mempersiapkan data set
dataset = pd.read_csv('train_preprocess.tsv.txt', sep="\t",header=None,names=["text","label"])
dataset

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [ ]:
dataset.isna().sum() #test 

normalisasi text

In [ ]:
def lowercasing(paragraph):
    return paragraph.lower()

def menghilangkan_tandabaca(paragraph):
    new_paragraph = re.sub(fr'[{punctuation}]', r'', paragraph)
    return new_paragraph

In [ ]:
def text_normalization(paragraph):
    paragraph = lowercasing(paragraph)
    paragraph = menghilangkan_tandabaca(paragraph)
    paragraph = re.sub(r"[ ]+",r' ',paragraph)
    return paragraph

membagi 3 data train split test

In [ ]:
#membagi 3 data train split test
train_data,test_data=train_test_split(dataset)
train_data['text'] = train_data['text'].apply(lambda x: text_normalization(x))
test_data['text'] = test_data['text'].apply(lambda x: text_normalization(x))
train_data, test_data = train_test_split(dataset, test_size=20)
train_data, val_data = train_test_split(train_data, test_size=20)
dataset.shape
dataset.text[1000]
dataset.label.value_counts()
data_preprocessed=dataset.text.tolist()

proses feature extraction

In [ ]:
#process feature extraction
count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)
X = count_vect.transform(data_preprocessed)
print("Feature extration selesai")

In [ ]:
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(data_preprocessed)
X = tfidf_vect.fit_transform(data_preprocessed)
print("Feature extration selesai")

In [ ]:
#import pickles
pickle.dump(count_vect, open("feature.p", "wb"))
classes = dataset.label
classes

X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

prepare train and extraction

In [ ]:
#menggunakan neural network
model = MLPClassifier()
model.fit(X_train, y_train)
print("Training selesai")

pickle.dump(model, open("model.p", "wb"))


model test ting and evaluation

In [ ]:
from sklearn.metrics import classification_report
test = model.predict(X_test)
print("Testing selesai")
print(classification_report(y_test,test))

In [ ]:
#training modeling & folding

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration , data in enumerate (kf.split(X), start=1):
  data_train = X[data[0]]
  target_train = y[data[0]]

  data_test = X[data[1]]
  target_test = y[data[1]]

  #clf = MultinomialNB()
  clf = MLPClassifier()

  clf.fit(data_train,target_train)

  preds = clf.predict(data_test)

  accuracy = accuracy_score(target_test,preds)

  print("Training ke-", iteration)
  print(classification_report(target_test, preds))
  print("===============================")

  accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)
print()
print()
print()
print("Rata-rata Accuracy", average_accuracy)


In [ ]:
class_report=classification_report(y_true=test_data[["label"]],y_pred=prediction,output_dict=True)

In [ ]:
formatted_report = {
    'precision': {},
    'recall': {},
    'f1-score': {},
    'support': {}
}

for class_label, metrics in class_report.items():
    if class_label in ['accuracy', 'macro avg', 'weighted avg']:
        continue
    formatted_report['precision'][class_label] = metrics['precision']
    formatted_report['recall'][class_label] = metrics['recall']
    formatted_report['f1-score'][class_label] = metrics['f1-score']
    formatted_report['support'][class_label] = metrics['support']

# Masukkan metrik keseluruhan
overall_metrics = class_report['macro avg']
formatted_report['precision']['macro avg'] = overall_metrics['precision']
formatted_report['recall']['macro avg'] = overall_metrics['recall']
formatted_report['f1-score']['macro avg'] = overall_metrics['f1-score']
formatted_report['support']['macro avg'] = overall_metrics['support']

# Masukkan metrik weighted average
weighted_metrics = class_report['weighted avg']
formatted_report['precision']['weighted avg'] = weighted_metrics['precision']
formatted_report['recall']['weighted avg'] = weighted_metrics['recall']
formatted_report['f1-score']['weighted avg'] = weighted_metrics['f1-score']
formatted_report['support']['weighted avg'] = weighted_metrics['support']

# Tambahkan accuracy
formatted_report['accuracy'] = class_report['accuracy']

# Ubah ke dalam format JSON
import json
formatted_report_json = json.dumps(formatted_report, indent=4)

# Simpan ke dalam file JSON
with open('formatted_classification_report.json', 'w') as json_file:
    json_file.write(formatted_report_json)


In [ ]:
import json
json.dump(report_cnn,open("json/report_cnn.json","w"),indent=4)